## **Trabajo Práctico Individual n° 2: Aprendizaje Supervisado**
____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Ciencia de Datos - Curso I5521__
____

- Nombre: Sol
- Apellido: Mazzeo
- Legajo: 1636145

En este trabajo práctico aplicaremos algunas técnicas aprendidas para el aprendizaje automático supervisado.

Utilizaremos un dataset de airbnb donde buscaremos predecir el precio de las noches de cada hogar en funcion de algunas variables.

#### Variables:
- Unnamed: 0: Variable de contador de registros (no será utilizada en nuetro análisis)
- neighbourhood: Barrio donde se encuentra ubicado la casa.
- room_type: Tipo de hogar en alquiler.
- price: Precio por noche.
- minimum_nights: Cantidad minimas de noches que pueden alquilar el hogar.
- number_of_reviews: Cantidad de reviews
- reviews_per_month: Tasa de reviews que tienen por mes
- calculated_host_listings_count: Cantidad de hogares ofrecidos por el host.
- availability_365: Cantidad de dias que se encuentra en alquiler el hogar

## **Objetivos del trabajo practico**
- resolver los requisitos que se detallan en las proximas celdas
- dejar comentarios # en cada linea de codigo que se agregue
- fecha limite de entrega: 13 de Octubre 2022 a las 23h59

In [1]:
## Importo librerías de maniupulación de datos
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
## Importo librerías de Aprendizaje automático
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

##### **Objetivo 1**
Lo primero que deberan hacer será importar
- el dataset bajo en nombre "london"
- corroborar que lo hayamos cargado correctamente.

In [3]:
#guardo el dataset en la variable "London"
london = pd.read_csv('TP2_airbnb_london.csv')
london.head()

,Unnamed: 0,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0,Haringey,Private room,35,1,131,1.71,4,262
1,1,Ealing,Private room,70,2,2,0.07,6,62
2,2,Islington,Private room,45,1,14,0.14,2,364
3,5,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,6,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 2**
Ahora imprimir también las dimensiones del dataset y eliminar la variable "Unnamed:0" del dataset.
Volver a guardar el dataset sin esta variable bajo el nombre "london" 

In [4]:
#Imprimo las dimensiones del dataset
np.shape(london)

(51827, 9)

In [5]:
#Elimino la variable "Unnamed: 0"
london = london.drop('Unnamed: 0', axis=1)

In [6]:
#Verifico haber sacado la columna
london.head()

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Haringey,Private room,35,1,131,1.71,4,262
1,Ealing,Private room,70,2,2,0.07,6,62
2,Islington,Private room,45,1,14,0.14,2,364
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295
4,Barnet,Private room,29,5,114,1.13,3,285


##### **Objetivo 3**
Corroborar que el dataset no cuenta con valores nulos, si los hubiera llenarlos todos con 0.

In [7]:
#Muestro la información del dataset
london.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51827 entries, 0 to 51826
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood                   51827 non-null  object 
 1   room_type                       51827 non-null  object 
 2   price                           51827 non-null  int64  
 3   minimum_nights                  51827 non-null  int64  
 4   number_of_reviews               51827 non-null  int64  
 5   reviews_per_month               51827 non-null  float64
 6   calculated_host_listings_count  51827 non-null  int64  
 7   availability_365                51827 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 3.2+ MB


In [8]:
#Chequeo que no haya ningún valor nulo
london.isnull().sum().sum()

0

In [9]:
col_nulls = london.isnull().any()
col_nulls
#No tiene nulos

neighbourhood                     False
room_type                         False
price                             False
minimum_nights                    False
number_of_reviews                 False
reviews_per_month                 False
calculated_host_listings_count    False
availability_365                  False
dtype: bool

##### **Objetivo 4**
Mostrar el tipo de dato para las variables "neighbourhood" y "room_type".

In [10]:
#Imprimo el data type de las variables "neighbourhood" y "room_type"
print(london.neighbourhood.dtypes)
print(london.room_type.dtypes)

object
object


##### **Objetivo 5**
Generar las variables dummies para estas dos variables categóricas y unirlas al dataset london.


In [11]:
#Genero las dummies para la variable neighbourhood, dentro de la variable neigh_dummies.
neigh_dummies = pd.get_dummies(pd.Series(london['neighbourhood']))

Imprimir las dimensiones y primeras lineas de estas dummies

In [12]:
#Imprimo las dimensiones de los dummies para neighbourhood
np.shape(neigh_dummies)

(51827, 33)

In [13]:
#Imprimo las primeras lineas del dummie
neigh_dummies.head()

,Barking and Dagenham,Barnet,Bexley,Brent,Bromley,Camden,City of London,Croydon,Ealing,Enfield,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Generamos las dummies para la variable room_type, dentro de la variable room_dummies.
room_dummies = pd.get_dummies(pd.Series(london['room_type']))

Imprimir las dimensiones y primeras lineas de estas dummies

In [15]:
#Imprimo las dimensiones de los dummies generados para room_types
np.shape(room_dummies)

(51827, 3)

In [16]:
#Imprimo las primeras lineas del dummie
room_dummies.head()

,Entire home/apt,Private room,Shared room
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


Joinear las variables dummies al dataset



In [17]:
#Hago un join con los dummies 
london = london.join(room_dummies).join(neigh_dummies)

In [18]:
#Uso solo 5000 lineas para no demorar el procesamiento
london=london.iloc[:5000]
london

,neighbourhood,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,Entire home/apt,Private room,...,Merton,Newham,Redbridge,Richmond upon Thames,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster
0,Haringey,Private room,35,1,131,1.71,4,262,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Ealing,Private room,70,2,2,0.07,6,62,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Islington,Private room,45,1,14,0.14,2,364,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Tower Hamlets,Entire home/apt,102,5,7,0.07,56,295,1,0,...,0,0,0,0,0,0,1,0,0,0
4,Barnet,Private room,29,5,114,1.13,3,285,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,Tower Hamlets,Entire home/apt,120,3,1,0.02,1,19,1,0,...,0,0,0,0,0,0,1,0,0,0
4996,Kensington and Chelsea,Entire home/apt,220,2,13,0.27,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4997,Hackney,Private room,62,2,139,2.91,1,330,0,1,...,0,0,0,0,0,0,0,0,0,0
4998,Camden,Entire home/apt,125,1,62,1.30,3,0,1,0,...,0,0,0,0,0,0,0,0,0,0


##### Dividiremos nuestro dataset en la variable dependiente "y" (label o etiqueta) y las variables independientes "x"

In [19]:
#Guardoel target en la variable dependiente "y" y las variables independientes "x"
y = np.array(london[["price"]])
x = london.drop(['price', 'neighbourhood','room_type'], axis=1)

##### **Objetivo 7**

Dividir el dataset entre Train y Test, tomar un test_size del 30%.

In [20]:
# Dividimos el dataset entre test y train
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

##### **Objetivo 8**
Con el StandardScaler generado en la linea siguiente, realizar el transform para el xtrain y xtest guardandolos en nuevas variables llamadas "xtrain_scal" y "xtest_scal".


In [21]:
#Separo los datasets de train y test de los dummies para hacer el scaling
x_train_nodummies = x_train.iloc[:,:5]
x_test_nodummies= x_test.iloc[:,:5]
#Los dummies los convierto en numpy arrays para poder usar np.concatenate 
x_train_dummies = x_train.iloc[:,5:].to_numpy()
x_test_dummies = x_test.iloc[:,5:].to_numpy()

In [22]:
#Ajustamos el scaler al dataset
scaler = preprocessing.StandardScaler().fit(x_train_nodummies)

In [23]:
#Scaling al dataset de train
xtrain_scal = scaler.transform(x_train_nodummies)

In [24]:
#Scaling al dataset de test
xtest_scal = scaler.transform(x_test_nodummies)

In [25]:
#Vuelvo a unir las features que les hice scaling con las dummies
xtrain_scal = np.concatenate((xtrain_scal,x_train_dummies),axis=1)
xtest_scal = np.concatenate((xtest_scal,x_test_dummies),axis=1)

##### **Objetivo 9**
Crear un modelo lineal y entrenarlo para nuestro dataset.

Obtener las predicciones y el "mean_squared_error" para el test set.

In [26]:
#Guardo el modelo lineal en la variable linear
linear = LinearRegression()

In [27]:
#Entreno al modelo con los datos de entrenamiento
linear.fit(xtrain_scal,y_train)

LinearRegression()

In [28]:
#Hago una predicción con los datos de test
prediction = linear.predict(xtest_scal)
print(prediction,y_test)

[[124.57041837]
 [113.41840421]
 [ 57.30863232]
 ...
 [141.38587274]
 [ 65.75120575]
 [ 63.02108289]] [[155]
 [115]
 [ 48]
 ...
 [ 95]
 [ 56]
 [ 60]]


In [29]:
#Mean squared error de las prediciones
mean_squared_error(y_test,prediction)

1502.4984578783221

##### **Objetivo 10**
Crear un SVM Regressor y un GridSearch para realizar la búsqueda de hiperparámetros.

Asignar dos posibles valores para el parámetro "kernel", y tres posibles valores para "C" y "gamma".

Setear al parámetro "refit" en True y hacer 5 CrossValidation.

In [30]:
#Selecciono el modelo Support Vector Regressor
model = SVR()

In [31]:
#Seteo los posibles valores del modelo
model_param = {'kernel':('linear', 'rbf'), 'C':[1, 10 , 20], 'gamma':[ 0.1,0.5,1]}

In [32]:
np.shape(xtrain_scal)

(3500, 41)

In [33]:
#Ejecuto un gridsearch para encontrar los mejores hiperparametros
regressor_svr = GridSearchCV(model, model_param, cv=5, verbose=True, refit = True)

In [34]:
#Fiteo el modelo a los datos de entrenamiento
regressor_svr.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [1, 10, 20], 'gamma': [0.1, 0.5, 1],
                         'kernel': ('linear', 'rbf')},
             verbose=True)

##### **Objetivo 11**

Mostrar cuales fueron los mejores hiperparámetros y para estos obtener el mean_squared_error en el test set.

In [35]:
#Imprimo los mejores hiperparametros
regressor_svr.best_params_

{'C': 20, 'gamma': 0.1, 'kernel': 'rbf'}

In [36]:
#Imprimo el mejor score
regressor_svr.best_score_

0.48783478990432094

In [37]:
#Realizo la predicción con los datos de test
prediction2 = regressor_svr.predict(xtest_scal)
prediction2

array([111.3586282 , 100.77647865,  43.86544795, ..., 121.42752755,
        51.77162273,  59.16774561])

In [38]:
mean_squared_error(y_test,prediction2)

1524.8910333286083

##### **Objetivo 12**
Repetir el objetivo 10 pero ahora seteando nuevos valores para los hiperparámetros mencionados con el objetivo de mejorar las predicciones.

Cuales son los mejores hiperparámetros ahora?
Obtenemos una mejora en las predicciones?
Mostrar los resultados.

In [39]:
#Selecciono el modelo Support Vector Regressor
model2 = SVR()

In [40]:
#Seteo los posibles valores del modelo
model_param2 = {'kernel':('poly', 'rbf'), 'C':[20, 30, 40], 'gamma':[ 0.01,0.05,0.1]}

In [41]:
#Ejecuto un gridsearch para encontrar los mejores hiperparametros
regressor_svr2 = GridSearchCV(model2, model_param2, cv=5, verbose=True, refit = True)

In [42]:
#Fiteo el modelo a los datos de entrenamiento
regressor_svr2.fit(xtrain_scal, y_train.ravel())

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [20, 30, 40], 'gamma': [0.01, 0.05, 0.1],
                         'kernel': ('poly', 'rbf')},
             verbose=True)

In [43]:
#Imprimo los mejores hiperparametros
regressor_svr2.best_params_

{'C': 40, 'gamma': 0.05, 'kernel': 'rbf'}

In [44]:
#Realizo la predicción con los datos de test
prediction3 = regressor_svr2.predict(xtest_scal)
prediction3

array([110.90780118, 100.29936651,  43.80702892, ..., 122.81607266,
        51.02642249,  58.18413373])

In [45]:
#Calculo el Mean Square Error
mean_squared_error(y_test,prediction3)

1517.778036625953